In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm

In [35]:
df = pd.read_csv('mRubis_Transition_Matrix.csv')
walk_matrix = df.to_numpy()[:,1:-1].astype(float) 
walk_matrix /= np.sum(walk_matrix, axis=1)
walk_matrix

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 1.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 1.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.25      , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.25      , 0.        , 0.        ,
        0.        , 0.25      , 0.25      , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.        , 1.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.   

In [36]:
status_transitions = np.array(
    [[0.8,0.1,0.1],
    [0.1,0.7,0.2],
    [0.4,0.1,0.5]]
)

component_transitions = np.array(
    [[0.3,0.7],
    [1.0,0.0]]
)

In [37]:
def print_state(current_component, current_status):
    status_descriptor = "S"
    status_mapping = {0: "o", 1: "d", 2:"u"}
    status_descriptor += status_mapping[current_status]
    status_descriptor += str(current_component)
    return status_descriptor

In [93]:
walk_length = 1000
num_walks = 1000

walks = []

for _ in tqdm(range(num_walks)):
    walk = []

    current_component = np.random.choice(range(19))
    current_status = np.random.choice(range(3))

    walk.append(print_state(current_component, current_status))

    for _ in range(walk_length-1):
        new_status = np.random.choice(range(3), p=status_transitions[current_status])

        no_status_change = int(current_status==new_status)
        stay_in_component = (1 == np.random.choice(range(2), p=component_transitions[no_status_change]))

        if stay_in_component:
            new_component = current_component
        else:
            new_component = np.random.choice(range(19), p=walk_matrix[current_component]/np.sum(walk_matrix[current_component]))

        current_status = new_status
        current_component = new_component

        walk.append(print_state(current_component, current_status))

    walks.append(walk)


100%|██████████| 1000/1000 [00:42<00:00, 23.72it/s]


In [94]:
transition_matrix = np.zeros((19*3, 19*3))
transition_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [95]:
for walk in walks:
    new_state = walk[0]

    reverse_status_mapping = {'o': 0, 'd': 1, 'u': 2}
    component = int(new_state[2:])
    status = reverse_status_mapping[new_state[1]]
    state = (3 * component) + status

    for new_state in walk[1:]:
        component = int(new_state[2:])
        
        status = reverse_status_mapping[new_state[1]]
        new_state = (3 * component) + status
        transition_matrix[state][new_state] += 1
        state = new_state

sums = np.sum(transition_matrix, axis=1)
sums[sums == 0] = 1
transition_matrix = (transition_matrix.T / sums ).T

In [96]:
#Steady state
val, vec  = np.linalg.eig(transition_matrix)
#find index of egenvalue 1
index = np.where(val - 1 < 0.001)[0][0]
steady_state = (vec.T[index] / np.sum(vec.T[index])).real
steady_state

array([0.01754386, 0.01754386, 0.01754386, 0.01754386, 0.01754386,
       0.01754386, 0.01754386, 0.01754386, 0.01754386, 0.01754386,
       0.01754386, 0.01754386, 0.01754386, 0.01754386, 0.01754386,
       0.01754386, 0.01754386, 0.01754386, 0.01754386, 0.01754386,
       0.01754386, 0.01754386, 0.01754386, 0.01754386, 0.01754386,
       0.01754386, 0.01754386, 0.01754386, 0.01754386, 0.01754386,
       0.01754386, 0.01754386, 0.01754386, 0.01754386, 0.01754386,
       0.01754386, 0.01754386, 0.01754386, 0.01754386, 0.01754386,
       0.01754386, 0.01754386, 0.01754386, 0.01754386, 0.01754386,
       0.01754386, 0.01754386, 0.01754386, 0.01754386, 0.01754386,
       0.01754386, 0.01754386, 0.01754386, 0.01754386, 0.01754386,
       0.01754386, 0.01754386])

In [97]:
#Question 1
# Formula for 2 components
# Sum for all Componens X: P(SoX) * P(Sd1 | SoX) * (Su1 | Sd1) * ( P(So1 | Su1) + P(So2 | Su1) ) + P(SoX) * P(Su1 | SoX) * (Sd1 | Su1) * ( P(So1 | Sd1) + P(So2 | Sd1) )

P_d_to_u = 0
for idx in range(19):
    idx *= 3
    P_SoX = steady_state[idx]
    P_SdX_SoX = transition_matrix[idx][1]
    P_SuX_SdX = transition_matrix[1][2]
    P_SoY_SuX = 0
    for idx_2 in range(19):
        idx_2 *= 3
        P_SoY_SuX += transition_matrix[2][idx_2]
    P_d_to_u += P_SoX * P_SdX_SoX * P_SuX_SdX * P_SoY_SuX

P_u_to_d = 0
for idx in range(19):
    idx *= 3
    P_SoX = steady_state[idx]
    P_SuX_SoX = transition_matrix[idx][2]
    P_SdX_SuX = transition_matrix[2][1]
    P_SoY_SdX = 0
    for idx_2 in range(19):
        idx_2 *= 3
        P_SoY_SdX += transition_matrix[1][idx_2]
    P_u_to_d += P_SoX * P_SuX_SoX * P_SdX_SuX * P_SoY_SdX

P = P_d_to_u + P_u_to_d
print(P)
    

0.00012797968339022848


In [98]:
# Question 2
# Cascade only on Component 2:

P_d_to_u = 0
for idx in range(19):
    idx *= 3
    P_SoX = steady_state[idx]
    P_SdX_SoX = transition_matrix[idx][3+1]
    P_SuX_SdX = transition_matrix[3+1][3+2]
    P_SoY_SuX = 0
    for idx_2 in range(19):
        idx_2 *= 3
        P_SoY_SuX += transition_matrix[3+2][idx_2]
    P_d_to_u += P_SoX * P_SdX_SoX * P_SuX_SdX * P_SoY_SuX

P_u_to_d = 0
for idx in range(19):
    idx *= 3
    P_SoX = steady_state[idx]
    P_SuX_SoX = transition_matrix[idx][3+2]
    P_SdX_SuX = transition_matrix[3+2][3+1]
    P_SoY_SdX = 0
    for idx_2 in range(19):
        idx_2 *= 3
        P_SoY_SdX += transition_matrix[3+1][idx_2]
    P_u_to_d += P_SoX * P_SuX_SoX * P_SdX_SuX * P_SoY_SdX

P_cascade_only_in_component_2 = P_d_to_u + P_u_to_d

# Cascade from any component to component 2:
P_X_to_2 = 0
for idx in range(19):
    if idx == 1:
        continue
    idx *= 3
    # P(SdX) * P(Sd2 | SdX) + P(SuX) * P(Sd2 | SuX) + P(SdX) * P(Su2 | SdX) + P(SuX) * P(Su2 | SuX)
    P_Sd2_SdX = steady_state[idx+1] * transition_matrix[idx+1][4]
    P_Su2_SdX = steady_state[idx+1] * transition_matrix[idx+1][5]
    P_Sd2_SuX = steady_state[idx+2] * transition_matrix[idx+2][4]
    P_Su2_SuX = steady_state[idx+2] * transition_matrix[idx+2][5]

    P_X_to_2 += P_Sd2_SdX + P_Su2_SdX + P_Sd2_SuX + P_Su2_SuX

# Cascade fomr component 2 to any other component

P_2_to_X = 0

for idx in range(19):
    if idx == 1:
        continue
    idx *= 3
    # P(SoX) * P(Sd2 | SoX) * (P(SdY | Sd2) + P(SuY | Sd2)) + 
    # P(SoX) * P(Su2 | SoX) * (P(SdY | Su2) + P(SuY | Su2)) + 
    # P(SoX) * P(Su2 | SoX) + P(Sd2 | Su2) * (P(SdY | Sd2) + P(SuY | Sd2)) + 
    # P(SoX) * P(Sd2 | SoX) + P(Su2 | Sd2) * (P(SdY | Su2) + P(SuY | Su2)) + 
    P_SoX = steady_state[idx]
    P_Sd2_SoX = transition_matrix[idx][4]
    P_Su2_SoX = transition_matrix[idx][5]
    P_Su2_Sd2 = transition_matrix[4][5]
    P_Sd2_Su2 = transition_matrix[5][4]
    P_SfY_Sd2 = 0
    P_SfY_Su2 = 0
    for idx_2 in range(19):
        if idx_2 == 1:
            continue
        idx_2 *= 3
        P_SfY_Sd2 += transition_matrix[4][idx_2+1] + transition_matrix[4][idx_2+2]
        P_SfY_Su2 += transition_matrix[5][idx_2+1] + transition_matrix[5][idx_2+2]

    P_2_to_X += P_SoX * P_Sd2_SoX * P_SfY_Sd2  +  P_SoX * P_Su2_SoX * P_SfY_Su2  +  P_SoX * P_Su2_SoX * P_Sd2_Su2 * P_SfY_Sd2  +  P_SoX * P_Sd2_SoX * P_Su2_Sd2 * P_SfY_Su2

P = P_cascade_only_in_component_2 + P_X_to_2 + P_2_to_X
print(P)


0.023368340936168647


In [99]:
# Question 3
# P(SdX) * P(SdY | SdX) * P(SoX | SdY) + P(SdX) * P(SuY | SdX) * P(SoX | SuY) + P(SuX) * P(SdY | SuX) * P(SoX | SdY) + P(SuX) * P(SuY | SuX) * P(SoX | SuY)
# For only unavailable
# P(SuX) * P(SuY | SuX) * P(SoX | SuY)

P_failure_masking = 0

for idx in range(19):
    idx *= 3
    #P_SdX = steady_state[idx+1]
    P_SuX = steady_state[idx+2]

    for idx_2 in range(19):
        idx_2 *= 3
        if idx_2 == idx:
            continue

        #P_SdY_SdX = transition_matrix[idx+1][idx_2+1]
        #P_SdY_SuX = transition_matrix[idx+2][idx_2+1]
        #P_SuY_SdX = transition_matrix[idx+1][idx_2+2]
        P_SuY_SuX = transition_matrix[idx+2][idx_2+2]

        #P_SoX_SdY = transition_matrix[idx_2+1][idx]
        P_SoX_SuY = transition_matrix[idx_2+2][idx]

        #P += P_SdX * P_SdY_SdX * P_SoX_SdY  +  P_SdX * P_SuY_SdX * P_SoX_SuY  +  P_SuX * P_SdY_SuX * P_SoX_SdY  +  P_SuX * P_SuY_SuX * P_SoX_SuY
        P_failure_masking +=  P_SuX * P_SuY_SuX * P_SoX_SuY

print(P_failure_masking)

0.0


In [100]:
# Question 4
#P(S_dX) * P(SdY | SdX)

P = 0

for idx in range(19):
    idx *= 3
    P_SdX = steady_state[idx+1]

    for idx_2 in range(19):
        idx_2 *= 3
        if idx_2 == idx:
            continue

        P_SdY_SdX = transition_matrix[idx+1][idx_2+1]

        P += P_SdX * P_SdY_SdX

print(P)

0.23401374651152745


In [101]:
# Question 5
# Sum of P(SoX) for all X

P = np.sum(steady_state[np.arange(19)*3])

print(P)

0.3333333333333334


In [102]:
# Question 6
# P(SoX) * ( P(SdX | SoX) * P(SoX | SdX)  +  P(SuX | SoX) * P(SoX | SuX))

P = 0
for n in range(1,50):
    for idx in range(19):
        idx *= 3
        P_SoX = steady_state[idx]

        P_SdX_SoX = transition_matrix[idx][idx+1]
        P_SuX_SoX = transition_matrix[idx][idx+2]

        P_SoX_SdX = transition_matrix[idx+1][idx]
        P_SoX_SuX = transition_matrix[idx+2][idx]

        P += P_SoX * (((P_SdX_SoX * P_SoX_SdX) ** n)  +  ((P_SuX_SoX * P_SoX_SuX) ** n))

print(P)

0.008268272564954248


In [103]:
# Question 7

# P = 0, because intermittent failure and failure cascade are disjoint events
0

0

In [92]:
# Question 8

# P(failure_masking | failure_cascade) = ( P( failure_cascade | failure_masking) * P(failure_masking)) / P(failure_cascade)
# Assumption: P( failure_cascade | failure_masking) = 1
# P(failure_masking | failure_cascade) = P(failure_masking) / P(failure_cascade)

P_failure_cascade = 0 
for component in range(19):
    P_d_to_u = 0
    for idx in range(19):
        idx *= 3
        P_SoX = steady_state[idx]
        P_SdX_SoX = transition_matrix[idx][3*component+1]
        P_SuX_SdX = transition_matrix[3+1][3*component+2]
        P_SoY_SuX = 0
        for idx_2 in range(19):
            idx_2 *= 3
            P_SoY_SuX += transition_matrix[3*component+2][idx_2]
        P_d_to_u += P_SoX * P_SdX_SoX * P_SuX_SdX * P_SoY_SuX

    P_u_to_d = 0
    for idx in range(19):
        idx *= 3
        P_SoX = steady_state[idx]
        P_SuX_SoX = transition_matrix[idx][3*component+2]
        P_SdX_SuX = transition_matrix[3*component+2][3*component+1]
        P_SoY_SdX = 0
        for idx_2 in range(19):
            idx_2 *= 3
            P_SoY_SdX += transition_matrix[3*component+1][idx_2]
        P_u_to_d += P_SoX * P_SuX_SoX * P_SdX_SuX * P_SoY_SdX

    P_cascade_only_in_component_2 = P_d_to_u + P_u_to_d

    # Cascade from any component to component 2:
    P_X_to_2 = 0
    for idx in range(19):
        if idx == component:
            continue
        idx *= 3
        # P(SdX) * P(Sd2 | SdX) + P(SuX) * P(Sd2 | SuX) + P(SdX) * P(Su2 | SdX) + P(SuX) * P(Su2 | SuX)
        P_Sd2_SdX = steady_state[idx+1] * transition_matrix[idx+1][3*component+1]
        P_Su2_SdX = steady_state[idx+1] * transition_matrix[idx+1][3*component+2]
        P_Sd2_SuX = steady_state[idx+2] * transition_matrix[idx+2][3*component+1]
        P_Su2_SuX = steady_state[idx+2] * transition_matrix[idx+2][3*component+2]

        P_X_to_2 += P_Sd2_SdX + P_Su2_SdX + P_Sd2_SuX + P_Su2_SuX

    # Cascade fomr component 2 to any other component

    P_2_to_X = 0

    for idx in range(19):
        if idx == component:
            continue
        idx *= 3
        # P(SoX) * P(Sd2 | SoX) * (P(SdY | Sd2) + P(SuY | Sd2)) + 
        # P(SoX) * P(Su2 | SoX) * (P(SdY | Su2) + P(SuY | Su2)) + 
        # P(SoX) * P(Su2 | SoX) + P(Sd2 | Su2) * (P(SdY | Sd2) + P(SuY | Sd2)) + 
        # P(SoX) * P(Sd2 | SoX) + P(Su2 | Sd2) * (P(SdY | Su2) + P(SuY | Su2)) + 
        P_SoX = steady_state[idx]
        P_Sd2_SoX = transition_matrix[idx][3*component+1]
        P_Su2_SoX = transition_matrix[idx][3*component+2]
        P_Su2_Sd2 = transition_matrix[3*component+1][3*component+2]
        P_Sd2_Su2 = transition_matrix[3*component+2][3*component+1]
        P_SfY_Sd2 = 0
        P_SfY_Su2 = 0
        for idx_2 in range(19):
            if idx_2 == component:
                continue
            idx_2 *= 3
            P_SfY_Sd2 += transition_matrix[3*component+1][idx_2+1] + transition_matrix[3*component+1][idx_2+2]
            P_SfY_Su2 += transition_matrix[3*component+2][idx_2+1] + transition_matrix[3*component+2][idx_2+2]

        P_2_to_X += P_SoX * P_Sd2_SoX * P_SfY_Sd2  +  P_SoX * P_Su2_SoX * P_SfY_Su2  +  P_SoX * P_Su2_SoX * P_Sd2_Su2 * P_SfY_Sd2  +  P_SoX * P_Sd2_SoX * P_Su2_Sd2 * P_SfY_Su2

    P_failure_cascade += P_cascade_only_in_component_2 + P_X_to_2 + P_2_to_X

P_fm_fc = P_failure_masking / P_failure_cascade

print(P_fm_fc)


0.0
